In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import re
import string
from sklearn.model_selection import GridSearchCV
np.random.seed(1)

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
ArSAS = pd.read_csv(r'/content/gdrive/MyDrive/ArSAS..txt',delimiter='\t')

In [4]:
data = ArSAS

"""**printing the first 3 rows of the data**"""

data.head(3)

,#Tweet_ID,Tweet_text,Topic,Sentiment_label,Sentiment_label_confidence,Speech_act_label,Speech_act_label_confidence
0,929241870508724224,المباراة القـادمة #غانا x #مصر الجولة الأخيرة ...,Event,Positive,0.38,Assertion,0.62
1,928942264583376897,هل هذه هي سياسة خارجيه لدوله تحترم نفسها والآخ...,Entity,Negative,1.00,Expression,0.68
2,928615163250520065,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت ب...,Event,Positive,0.69,Assertion,1.00


In [5]:
print("On a  {} doublons dans Data.".format(data.duplicated().sum()))

data.drop_duplicates(inplace = True)

print("On a  {} doublons dans Data.".format(data.duplicated().sum()))

On a  26 doublons dans Data.
On a  0 doublons dans Data.


In [6]:

data.head(3)
df=data


In [7]:
def preprocessing(x):
    x = re.sub('@[^\s]+', ' ', x)
    x = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',x)

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030""]+", flags=re.UNICODE)
    emoji_pattern.sub(r'', x)

    ar_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ#'''
    en_punctuations = string.punctuation
    punctuations = ar_punctuations + en_punctuations
    x = x.translate(str.maketrans('', '', punctuations))

    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    x = re.sub(arabic_diacritics, '', str(x)) 

#     x = re.sub("[إأآا]", "ا", x)
#     x = re.sub("ى", "ي", x)
#     x = re.sub("ة", "ه", x)
#     x = re.sub("گ", "ك", x)
#     x = re.sub(r'(.)\1+', r'\1', x)

    return x


In [8]:
data["Clean_tweets"] = data.Tweet_text.apply(lambda x: preprocessing(str(x)))

In [9]:
data["Topics"] = data['Topic'].astype("category").cat.codes

In [10]:
data.Topics

0        1
1        0
2        1
3        1
4        1
        ..
19892    1
19893    1
19894    1
19895    1
19896    2
Name: Topics, Length: 19871, dtype: int8

In [11]:
X = data.Clean_tweets
y = data.Topics

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state = 42)

"""**printing the number of the train set and the test set**"""

print('Train set', X_train.shape)
print('Test set', X_test.shape)


Train set (15896,)
Test set (3975,)


In [13]:
featurizer = CountVectorizer(decode_error='ignore')
Xtrain = featurizer.fit_transform(X_train)
Xtest = featurizer.transform(X_test)


In [14]:
model = MultinomialNB()
model.fit(Xtrain, y_train)
print("train acc:", model.score(Xtrain, y_train))
print("test acc:", model.score(Xtest, y_test))

train acc: 0.9876069451434323
test acc: 0.9509433962264151
